In [1]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math 
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")
from IPython. display import clear_output

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/fb3-my-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/fb3-my-pip-wheels tokenizers')
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
%env TOKENIZERS_PARALLELISM=false

clear_output()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
print('device:', device)

tokenizers.__version__: 0.12.1
transformers.__version__: 4.20.1
device: cuda


In [2]:
BASE_PATH = '/kaggle/input/feedback-prize-english-language-learning'
SUBMISSION_PATH = os.path.join(BASE_PATH, 'sample_submission.csv')
TRAIN_PATH = os.path.join(BASE_PATH, 'train.csv')
TEST_PATH = os.path.join(BASE_PATH, 'test.csv')

TARGET_COLS = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

In [3]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
    
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features, *args):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector
    
class MaxPooling(nn.Module):
    def __init__(self):
        super(MaxPooling, self).__init__()
    
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        last_hidden_state_masked = last_hidden_state
        last_hidden_state_masked[input_mask_expanded == 0] = -1e-9 
        max_embeddings = torch.max(last_hidden_state_masked, 1)[0]
        return max_embeddings
    
class LSTMPooling(nn.Module):
    def __init__(self, num_layers, hidden_size, hiddendim_lstm):
        super(LSTMPooling, self).__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_lstm = hiddendim_lstm
        self.lstm = nn.LSTM(self.hidden_size, self.hiddendim_lstm, batch_first=True)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, all_hidden_states):
        # only use first and last
        hidden_states = torch.stack([
            all_hidden_states[layer_i][:, 0].squeeze()
            for layer_i in (-1, 0)],
            dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)
        out, _ = self.lstm(hidden_states, None)
        #out = self.dropout(out[:, -1, :])
        out = self.dropout(out.mean(dim=1))
        return out

class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 9, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average

class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
        nn.Linear(in_dim, in_dim),
        nn.LayerNorm(in_dim),
        nn.GELU(),
        nn.Linear(in_dim, 1),
        )

    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state).float()
        w[attention_mask==0]=float('-inf')
        w = torch.softmax(w,1)
        attention_embeddings = torch.sum(w * last_hidden_state, dim=1)
        return attention_embeddings

class MultiSampleDropout(nn.Module):
    def __init__(self, fc, num_dropout, prob_dropout):
        super(MultiSampleDropout, self).__init__()
        self.dropout = nn.Dropout
        self.num_dropout = num_dropout
        self.prob_dropout = prob_dropout
        self.classifier = fc
    def forward(self, out):
        if not type(self.prob_dropout) in [float, int]:            
            fcs = [self.classifier(self.dropout(p)(out)) for p in self.prob_dropout]
        else:
            fcs = [self.classifier(self.dropout(self.prob_dropout)(out)) for _ in range(self.num_dropout)]
        
        return torch.mean(torch.stack(fcs, dim=0), dim=0)

# ====================================================
# Model classes
# ====================================================
class FB3Model(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg 
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            # Turn off dropouts.
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            #LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        
        if pretrained:
            self.deberta_v3 = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.deberta_v3 = AutoModel.from_config(self.config)

        #if self.cfg.reinit_last_layer:
        #    # Re-init last layer of deberta.
        #    for module in self.deberta_v3.encoder.layer[-1:].modules():
        #        self._init_weights(module)
        # self.deberta_v3.gradient_checkpointing_enable()

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            #nn.init.xavier_uniform_(module.weight.data, gain=1.0)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

class WMPoolModel(FB3Model):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__(cfg, config_path=config_path, pretrained=pretrained)

        # Poolings.
        self.mean_head = MeanPooling()
        self.wpool_head = WeightedLayerPooling(self.config.num_hidden_layers, layer_start=12)

        self.fc_out = nn.Linear(self.config.hidden_size, cfg.num_target)
        self._init_weights(self.fc_out)
        
        self.layer_norm = nn.LayerNorm(self.config.hidden_size)
        self.qa_output = torch.nn.Linear(self.config.hidden_size, 2)
        self.attention_head = AttentionHead(self.config.hidden_size*4, self.config.hidden_size)
        
    def forward(self, x):
        pt_out = self.deberta_v3(**x)
        all_hidden_states = torch.stack(pt_out.hidden_states)
        # Weighted pooling of last n layers.
        logits = self.wpool_head(all_hidden_states)[:, 0] # Bx768
        y_hat = self.fc_out(logits)
        return y_hat

class MultiPoolModel(FB3Model):
    def __init__(self, cfg, config_path=None, pretrained=False, pool='mean'):
        super().__init__(cfg, config_path=config_path, pretrained=pretrained)
        
        # Define model layers.
        self.pool_name = cfg.pool_head
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.num_target)
        if cfg.pool_head in ['mean', 'attention', 'weighted']:
            self.pool = self._pool_layer(cfg.pool_head)
        elif '-' in cfg.pool_head:
            pools = cfg.pool_head.split('-')
            self.pool = nn.ModuleList([])
            for pool_ in pools:
                self.pool.append(self._pool_layer(pool_))
            self.fc = nn.Linear(self.config.hidden_size * len(self.pool), self.cfg.num_target)
        self._init_weights(self.fc)
        
        # Multi-sample dropout.
        self.multi_dropout = MultiSampleDropout(self.fc, self.cfg.num_dropout, self.cfg.prob_dropout)
    
    def _pool_layer(self, pool_name):
        assert pool_name in ['mean', 'attention', 'weighted']
        if pool_name == 'mean':
            pool = MeanPooling()
        elif pool_name == 'attention':
            pool = AttentionHead(self.config.hidden_size, self.config.hidden_size)
        elif pool_name == 'weighted':
            pool = WeightedLayerPooling(
                self.config.num_hidden_layers, 
                layer_start=9,
                layer_weights=None)
        return pool
    
    def _pool_feature(self, pool, pool_name, pt_outputs, attention_mask):
        assert pool_name in ['mean', 'attention', 'weighted']
        last_hidden_state = pt_outputs.last_hidden_state #batch_size x max_len x hidden_size
        all_hidden_states = torch.stack(pt_outputs.hidden_states) #num_layer x batch_size x max_len x hidden_size
        
        if pool_name == 'mean':
            pool_feature = pool(last_hidden_state, attention_mask)
        elif pool_name == 'attention':
            pool_feature = pool(last_hidden_state)
        elif pool_name == 'weighted':
            # Take the CLS token only.
            pool_feature = pool(all_hidden_states)[:, 0]
        return pool_feature

    def feature(self, x):
        pt_outputs = self.deberta_v3(**x)
        
        # Pooling feat.
        if type(self.pool) == nn.ModuleList:
            pool_features = []
            pool_names = self.pool_name.split('-')
            
            for pool_name, pool in zip(pool_names, self.pool):
                pool_features.append(self._pool_feature(pool, pool_name, pt_outputs, x['attention_mask']))
            pool_features = torch.cat(pool_features, dim=1)
        else:
            pool_features = self._pool_feature(self.pool, self.pool_name, pt_outputs, x['attention_mask'])
        return pool_features
    
    def forward(self, x):
        feature = self.feature(x)
        if self.cfg.use_dropout and self.training:
            y_hat = self.multi_dropout(feature)
        else:
            y_hat = self.fc(feature)
        return y_hat


class Attention4Model(FB3Model):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__(cfg, config_path=config_path, pretrained=pretrained)
        
        self.head = AttentionHead(self.config.hidden_size*4, self.config.hidden_size)
        self.fc_out = nn.Linear(self.config.hidden_size*4*2, self.cfg.num_target)
        self._init_weights(self.fc_out)

    def forward(self, x):
        pt_out = self.deberta_v3(**x)
        
        all_hidden_states = torch.stack(pt_out.hidden_states)
        cat_over_last_layers = torch.cat(
            (all_hidden_states[-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]),
            -1)
        # [CLS] embedding.
        cls_pooling = cat_over_last_layers[:, 0]   
        # Concat of 4 last layers.
        head_logits = self.head(cat_over_last_layers)

        if self.cfg.use_dropout and self.training:
            y_hat = self.multi_dropout(torch.cat([head_logits, cls_pooling], -1))
        else:
            y_hat = self.fc_out(torch.cat([head_logits, cls_pooling], -1))

        return y_hat
    
class AttentionModel(FB3Model):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__(cfg, config_path=config_path, pretrained=pretrained)

        # Poolings.
        self.att = nn.Sequential(
            nn.Linear(self.config.hidden_size, self.cfg.ap_hidden_size),
            nn.Tanh(),
            nn.Linear(self.cfg.ap_hidden_size, 1),
            nn.Softmax(dim=1),
        )
        self._init_weights(self.att)
        
        self.fc_out = nn.Linear(self.config.hidden_size, cfg.num_target)
        self._init_weights(self.fc_out)

    def forward(self, x):
        pt_out = self.deberta_v3(**x)
        last_hidden_states = pt_out.last_hidden_state
        att_weights = self.att(last_hidden_states)
        logits =  torch.sum(att_weights * last_hidden_states, dim=1)
        
        y_hat = self.fc_out(logits)
        return y_hat
    
class MeanModel(FB3Model):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__(cfg, config_path=config_path, pretrained=pretrained)

        # Poolings.
        self.mean_head = MeanPooling()

        # Head.
        self.fc_out = nn.Linear(self.config.hidden_size, cfg.num_target)
        self._init_weights(self.fc_out)
    
    def forward(self, x):
        pt_out = self.deberta_v3(**x)
        # Mean pooling.
        logits = self.mean_head(pt_out.last_hidden_state, x['attention_mask'])
        y_hat = self.fc_out(logits)
        return y_hat

######################################

class FB3Model(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg 
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            # Turn off dropouts.
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            #LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)

        #if self.cfg.reinit_last_layer:
        #    # Re-init last layer of deberta.
        #    for module in self.model.encoder.layer[-1:].modules():
        #        self._init_weights(module)
        #self.model.gradient_checkpointing_enable()

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

class WeightedAttentionModel(FB3Model):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__(cfg, config_path=config_path, pretrained=pretrained)

        self.weighted_pool = WeightedLayerPooling(
            self.config.num_hidden_layers, layer_start=9, layer_weights=None)
        self.att_pool = AttentionPooling(self.config.hidden_size)

        self.fc_out = nn.Linear(self.config.hidden_size*2, cfg.num_target)
        self._init_weights(self.fc_out)

    def forward(self, x):
        pt_out = self.model(**x)
        hidden_states = pt_out.hidden_states
        last_hidden_state = pt_out.last_hidden_state

        x1 = self.weighted_pool(torch.stack(hidden_states))[:, 0]
        x2 = self.att_pool(last_hidden_state, x['attention_mask'])

        y_hat = self.fc_out(torch.cat([x1, x2], dim=1))
        return y_hat
    
########################

class WeightedLayerPooling_(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layers = None, layer_weights = None):
        super(WeightedLayerPooling_, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        
        if layers:
            self.layer_weights = layer_weights if layer_weights is not None \
                else nn.Parameter(
                    torch.tensor([1] * len(layers), dtype=torch.float)
                )
            self.layers = layers
        else:
            self.layer_weights = layer_weights if layer_weights is not None \
                else nn.Parameter(
                   torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
                )
            self.layers = list(range(layer_start, num_hidden_layers+1))
            

    def forward(self, ft_all_layers):
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layers, :, :, :]
        
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average
    
class CustomModel(nn.Module):

    def __init__(self, CFG, config_path = None, pretrained = False):
        super().__init__()
        self.CFG = CFG
        self.config = AutoConfig.from_pretrained(config_path, output_hidden_states=True)
        self.model = AutoModel.from_config(self.config)
        self.pretrained = pretrained
                        
        fc_hidden_size = self.config.hidden_size
        if CFG.pooling == 'mean':
            self.pool = MeanPooling()
        elif CFG.pooling == 'max':
            self.pool = MaxPooling()
        elif CFG.pooling == 'min':
            self.pool = MinPooling()
        elif CFG.pooling == 'attention':
            self.pool = AttentionPooling(self.config.hidden_size)
        elif CFG.pooling == 'weightedlayer':
            self.pool = WeightedLayerPooling_(self.config.num_hidden_layers, layer_start = CFG.layer_start, layer_weights = None)
        elif CFG.pooling == 'weightedlayer-mean':
            self.pool = WeightedLayerPooling_(self.config.num_hidden_layers, layer_start = CFG.layer_start, layer_weights = None)
            self.mean_pool = MeanPooling()
        elif self.CFG.pooling == 'attention4':
            self.pool = AttentionHead(fc_hidden_size*4, 512)
            fc_hidden_size = fc_hidden_size*8
        self.fc = nn.Linear(fc_hidden_size, 6)
                        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        if self.CFG.pooling == 'attention4':
            all_layer_embeddings = torch.stack(outputs.hidden_states)
            cat_over_last_layers = torch.cat((all_layer_embeddings[-1], all_layer_embeddings[-2], all_layer_embeddings[-3], all_layer_embeddings[-4]), -1)
            cls_pooling = cat_over_last_layers[:, 0]
            head_logits = self.pool(cat_over_last_layers)
            feature = torch.cat([head_logits, cls_pooling], -1)
        elif self.CFG.pooling == 'weightedlayer':
            all_layer_embeddings = outputs[1]
            feature = self.pool(all_layer_embeddings)[:, 0]
        elif self.CFG.pooling == 'weightedlayer-mean':
            all_layer_embeddings = outputs[1]
            feature = self.pool(all_layer_embeddings)
            feature = self.mean_pool(feature, inputs['attention_mask'])
        else:
            last_hidden_states = outputs[0]
            feature = self.pool(last_hidden_states, inputs['attention_mask'])    
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output
    
##############################

class MeanAttentionModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg 
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            # Turn off dropouts.
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            #LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        
        if pretrained:
            self.deberta_v3 = AutoModel.from_pretrained(cfg.model, config=self.config)
            # Expand embedding dim for new tokens.
            self.deberta_v3.resize_token_embeddings(len(cfg.tokenizer))
        else:
            self.deberta_v3 = AutoModel.from_config(self.config)
            
        self.deberta_v3.gradient_checkpointing_enable()
        
        # Define model layers.
        self.fc = nn.Linear(self.config.hidden_size, 6)

        if cfg.pool == 'mean':
            self.pool = MeanPooling()
        elif cfg.pool == 'attention':
            self.pool = AttentionHead(self.config.hidden_size, self.config.hidden_size)
        elif cfg.pool == 'mean-attention':
            self.pool = nn.ModuleList([
                MeanPooling(),
                AttentionHead(self.config.hidden_size, self.config.hidden_size)
            ])
            self.fc = nn.Linear(self.config.hidden_size * len(self.pool), 6)
        elif cfg.pool == 'mean-attention-with-mask':
            self.pool = nn.ModuleList([
                MeanPooling(),
                AttentionPooling(self.config.hidden_size)
            ])
            self.fc = nn.Linear(self.config.hidden_size * len(self.pool), 6)
        # Re-init weights.
        self._init_weights(self.fc)
        
        # Multi-sample dropout.
        self.multi_dropout = MultiSampleDropout(self.fc, cfg.num_dropout, cfg.prob_dropout)
        
    def global_avg_pool(x):
        return torch.mean(x.view(x.size(0), x.size(1), -1), dim=-1)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def feature(self, x):
        pt_outputs = self.deberta_v3(**x)
        last_hidden_states = pt_outputs[0] # N x max_len x 768
        # Pooling feat.
        if type(self.pool) == nn.ModuleList:
            pool_feature = [pool(last_hidden_states, x['attention_mask']) for pool in self.pool]
            pool_feature = torch.cat(pool_feature, dim=1)
        else:
            pool_feature = self.pool(last_hidden_states, x['attention_mask']) # N x 768
        return pool_feature
    
    def forward(self, x, y=None, loss_fn=None):
        feature = self.feature(x)
        # if self.training:
        #    out = self.multi_dropout(feature)
        # else:
        #    out = self.fc(feature)
        out = self.fc(feature)
        return out

In [4]:
class Config(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
    def init(self, kwargs):
        super().init(kwargs)

        for k, v in kwargs.items():
            setattr(self, k, v)

    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

In [5]:
def seed_everything(seed=42):
    '''
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.
    '''
    random.seed(seed)
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # When running on the CuDNN backend, two further options must be set
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
seed_everything(seed=42)

def mc_rmse(y_true, y_pred):
    scores = []
    ncols = y_true.shape[1]
    
    for n in range(ncols):
        yn_true = y_true[:, n]
        yn_pred = y_pred[:, n]
        rmse_ = mean_squared_error(yn_true, yn_pred, squared=False)
        scores.append(rmse_)
    score = np.mean(scores) 
    return score, scores

def get_result(cfg, oof_df):
    labels = oof_df[TARGET_COLS].values
    preds = oof_df[[f"pred_{c}" for c in TARGET_COLS]].values
    score, scores = mc_rmse(labels, preds)
    print(f'score: {score:<.6f}  scores: {scores}')

In [6]:
def encode_text(cfg, text):
    if cfg.pretrained:
        inputs = cfg.tokenizer(
            text,
            None,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=cfg.max_len,
            return_token_type_ids=True,
            return_tensors='pt'
        )
        inputs = {k:v.squeeze(0) for k,v in inputs.items()}
    else:
        if 'roberta' in cfg.model:
            inputs = cfg.tokenizer.encode_plus(
                text,
                return_tensors = None,
                add_special_tokens = True,
                max_length = cfg.max_len,
                pad_to_max_length = True,
                truncation = True)
        elif '512' in cfg.name or '768' in cfg.name or '1024' in cfg.name:
            inputs = cfg.tokenizer.encode_plus(
                text,
                return_tensors = None,
                add_special_tokens = True,
                max_length = cfg.max_len,
                truncation = True)
        else:
            inputs = cfg.tokenizer.encode_plus(
                text, 
                return_tensors=None, 
                add_special_tokens=True)
        for k, v in inputs.items():
            inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs 

def preprocess(texts):
    texts = (
        texts
        .str.replace(r'\r\n', '<newline>', regex=True)
        .str.replace(r'\n', '<newline>', regex=True)
        .str.replace('<newline><newline>', '<newline>', regex=False)
        .values 
    )
    return texts

class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        if not cfg.pretrained and cfg.version in ['1', 'mean-attention']:
        #if not pretrained:
            print('preprocess')
            self.texts = preprocess(df['full_text'])
        else:
            self.texts = df['full_text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = encode_text(self.cfg, self.texts[item])
        return inputs

class TrainDataset(Dataset):
    def __init__(self, cfg, data):
        self.cfg = cfg
        self.xs = data['full_text']
        self.ys = data[TARGET_COLS].values 
        
    def __len__(self):
        return len(self.xs)
    
    def __getitem__(self, idx):
        x = encode_text(self.cfg, self.xs[idx])
        y = torch.tensor(self.ys[idx], dtype=torch.float32)
        return x, y

def create_data_loaders(cfg, folds, fold):
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    train_dataset = TrainDataset(cfg, train_folds)
    valid_dataset = TrainDataset(cfg, valid_folds)
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.batch_size,
        shuffle=True,
        num_workers=cfg.num_workers,
        pin_memory=True,
        drop_last=True
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False
    )
    return train_loader, valid_loader

In [7]:
def load_config(input_path, inference_weight=1):
    # Load CFG class.
    cfg = Config(**json.load(open(os.path.join(input_path, 'CFG.json'), 'r')))
    cfg.path = input_path
    cfg.config_path = os.path.join(cfg.path, 'config.pth')
    # Load tokenizer.
    tokenizer = AutoTokenizer.from_pretrained(os.path.join(cfg.path, 'tokenizer'))
    cfg.tokenizer = tokenizer
    
    cfg.inference_weight = inference_weight
    return cfg

def load_model(cfg, fold, version='1', **model_kwargs):
    # Load torch model.
    if version == '1':
        model = MultiPoolModel(cfg, config_path=cfg.config_path, pretrained=False, **model_kwargs)
    elif version == '2':
        model = Attention4Model(cfg, config_path=cfg.config_path, pretrained=False, **model_kwargs)
    elif version == '21':
        model = WMPoolModel(cfg, config_path=cfg.config_path, pretrained=False, **model_kwargs)
    elif version == 'mean':
        model = MeanModel(cfg, config_path=cfg.config_path, pretrained=False, **model_kwargs)
    elif version == 'attention':
        model = AttentionModel(cfg, config_path=cfg.config_path, pretrained=False, **model_kwargs)
    elif version == 'mix':
        model = MixModel(cfg, config_path=cfg.config_path, pretrained=False, **model_kwargs)
    elif version == 'weighted-attention':
        model = WeightedAttentionModel(cfg, config_path=cfg.config_path, pretrained=False, **model_kwargs)
    elif version == 'custom':
        model = CustomModel(cfg, config_path=cfg.config_path, pretrained=False, **model_kwargs)
    elif version == 'mean-attention':
        model = MeanAttentionModel(cfg, config_path=cfg.config_path, pretrained=False)
    state = torch.load(
        os.path.join(cfg.path, f"{cfg.model.replace('/', '-')}_fold{fold}_best.pth"),
        map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    return model

In [8]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state.detach().cpu()
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    #tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in test_loader:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

class Inferencer:
    def __init__(self, input_path=None, cfg=None, inference_weight=1):
        if cfg == None:
            self.cfg = load_config(input_path, inference_weight)
        else:
            self.cfg = cfg
    
    def predict(self, test_loader, device, stat_fn=np.mean):
        preds = []
        
        for fold in self.cfg.trn_fold:
            start = time.time()
            print('#'*10, cfg.path, '#'*10)
            
            print(f'Predicting fold {fold}...')
            model = load_model(self.cfg, fold, version=self.cfg.version)
            pred = inference_fn(test_loader, model, device)
            preds.append(pred)
            del model, pred; gc.collect()
            torch.cuda.empty_cache()
            
            end = time.time() - start
            print('#'*10, f'ETA: {end:.2f}s', '#'*10, '\n')
        
        
        self.preds = stat_fn(preds, axis=0) 
        self.preds = np.clip(self.preds, 1, 5)
        return self.preds
    
    def get_oof_result(self, file_type='pkl'):
        return get_result(self.cfg, self.get_oof_df(file_type))
    
    def get_oof_df(self, file_type='pkl'):
        if file_type == 'pkl':
            return pd.read_pickle(os.path.join(cfg.path, 'oof_df.pkl'))
        return pd.read_csv(os.path.join(cfg.path, 'oof_df.csv'))

# Fine-tuned models

In [9]:
v114_CFG = Config(
    model="microsoft/deberta-v3-base",
    version='1',
    num_target = 6,
    reinit_last_layer=True,
    reinit_fc=True,
    weight_decay=0.01,
    learning_rate=2e-5,
    layerwise_learning_rate_decay=1.5,
    use_dropout=False,
    prob_dropout=[0.06, 0.08, 0.1, 0.12, 0.14],
    num_dropout=5,
    pool_head='mean-attention',
    seed=42,
    n_fold=4,
    trn_fold=[0,1,2,3],
    path='../input/fb3models/v114/',
    config_path='../input/fb3models/v114/config.pth',
    tokenizer=AutoTokenizer.from_pretrained('../input/fb3models/v114/tokenizer')
)

weightedpool_CFG = Config(
    model='microsoft/deberta-v3-base',
    name='weighted_pool',
    version='1',
    num_target=6,
    reinit_last_layer=True,
    reinit_fc=True,
    weight_decay=0.01,
    learning_rate=1.5e-5,
    layerwise_learning_rate_decay=1.5,
    use_dropout=False,
    prob_dropout=[0.06, 0.08, 0.1, 0.12, 0.14],
    num_dropout=5,
    pool_head='weighted',
    seed=42,
    n_fold=4,
    trn_fold=[0,1,2,3],
    train=True,
    path='../input/fb3-train/',
    config_path='../input/fb3-train/config.pth',
    tokenizer=AutoTokenizer.from_pretrained('../input/fb3-train/tokenizer'),
    inference_weight=0.1)

v116_CFG = load_config('../input/fb3-colab-models/v116', inference_weight=0.1)
v116_CFG.path = '../input/fb3models/v116'
v116_CFG.config_path = '../input/fb3models/v116/config.pth'
v116_CFG.version = '1'

In [10]:
v112_CFG = Config(
    num_workers=1,
    batch_size=3,
    max_len=512,
    model="microsoft/deberta-v3-base",
    name='v112',
    version='1',
    num_target = 6,
    reinit_last_layer=True,
    reinit_fc=True,
    weight_decay=0.01,
    learning_rate=2e-5,
    layerwise_learning_rate_decay=1.5,
    use_dropout=False,
    prob_dropout=[0.06, 0.08, 0.1, 0.12, 0.14],
    num_dropout=5,
    pool_head='attention',
    seed=42,
    n_fold=4,
    trn_fold=[0,1,2,3],
    train=True,
    path='../input/fb3models/v112/',
    config_path='../input/fb3models/v112/config.pth',
    inference_weight=1.0,
    tokenizer=AutoTokenizer.from_pretrained('../input/fb3models/v112/tokenizer')
)

#####
v2_CFG = load_config('../input/fb3models/v2/', inference_weight=1.0)
v2_CFG.name = 'v2'
v2_CFG.version = '2'
v2_CFG.trn_fold = [0,1,2,3]

#####
v21_CFG = load_config('../input/fb3models/v21/', inference_weight=1)
v21_CFG.name = 'v21'
v21_CFG.version = '21'

#####
attention_fgm_CFG = load_config('../input/fb3models/20221114-192943-deberta-v3-base/', inference_weight=1.0)
attention_fgm_CFG.name = 'attention_fgm'
attention_fgm_CFG.version = 'custom'
attention_fgm_CFG.config_path = '../input/fb3models/20221114-192943-deberta-v3-base/config/config.json'

weighted_fgm_CFG = Config(
    pretrained=False,
    path='../input/fb3models/20221115-061243-deberta-v3-base',
    config_path='../input/fb3models/20221115-061243-deberta-v3-base/config/config.json',
    tokenizer=AutoTokenizer.from_pretrained('../input/fb3models/20221115-061243-deberta-v3-base/tokenizer'),
    name='weighted_fgm',
    version='custom',
    train = True,
    debug = False,
    offline = False,
    models_path = 'FB3-models',
    epochs = 5,
    save_all_models = False,
    competition = 'FB3',
    apex = True,
    print_freq = 20,
    num_workers = 4,
    model = 'microsoft/deberta-v3-base', #If you want to train on the kaggle platform, v3-base is realistic. v3-large will time out.
    loss_func = 'SmoothL1', # 'SmoothL1', 'RMSE'
    gradient_checkpointing = True,
    scheduler = 'cosine',
    batch_scheduler = True,
    num_cycles = 0.5,
    num_warmup_steps = 0,
    encoder_lr = 2e-5,
    decoder_lr = 2e-5,
    min_lr = 1e-6,
    #Layer-Wise Learning Rate Decay
    llrd = True,
    layerwise_lr = 5e-5,
    layerwise_lr_decay = 0.9,
    layerwise_weight_decay = 0.01,
    layerwise_adam_epsilon = 1e-6,
    layerwise_use_bertadam = False,
    #pooling
    pooling = 'weightedlayer', # mean, max, min, attention, weightedlayer
    layer_start = 11,
    layers=None,
    #init_weight
    init_weight = 'normal', # normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal
    #re-init
    reinit = True,
    reinit_n = 1,
    #adversarial
    fgm = True,
    awp = False,
    adv_lr = 1,
    adv_eps = 0.2,
    unscale = False,
    eps = 1e-6,
    betas = (0.9, 0.999),
    max_len = 512,
    weight_decay = 0.01,
    gradient_accumulation_steps = 1,
    max_grad_norm = 1000,
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
    seed = 42,
    cv_seed = 42,
    n_fold = 4,
    trn_fold = [0,1,2,3],
    batch_size = 8,
    n_targets = 6,
    gpu_id = 0) 

weighted_attention_CFG = load_config('../input/fb3models/weighted_attention_v3', inference_weight=1.0)
weighted_attention_CFG.name = 'weighted_attention'
weighted_attention_CFG.version = 'weighted-attention'

mean_attention_no_fgm_CFG = load_config('../input/fb3models/20221117-183420-deberta-v3-base-mean-attention-with-mask', inference_weight=1.0)
mean_attention_no_fgm_CFG.name = 'mean_attention_no_fgm'
mean_attention_no_fgm_CFG.version = 'mean-attention'

attention_large_fgm_CFG = load_config('../input/fb3models/20221118-164148-deberta-v3-large-attention_fgm')
attention_large_fgm_CFG.name = 'attention_large_fgm'
attention_large_fgm_CFG.version = 'custom'
attention_large_fgm_CFG.config_path = '../input/fb3models/20221118-164148-deberta-v3-large-attention_fgm/config/config.json'

attention_fgm_512_CFG = load_config('../input/fb3models/20221121-143655-deberta-v3-base-attention_fgm_512')
attention_fgm_512_CFG.name = 'attention_fgm_512'
attention_fgm_512_CFG.version = 'custom'
attention_fgm_512_CFG.config_path = '../input/fb3models/20221121-143655-deberta-v3-base-attention_fgm_512/config/config.json'

attention_fgm_768_CFG = load_config('../input/fb3models/20221120-072218-deberta-v3-base-attention_fgm')
attention_fgm_768_CFG.name = 'attention_fgm_768'
attention_fgm_768_CFG.version = 'custom'
attention_fgm_768_CFG.config_path = '../input/fb3models/20221120-072218-deberta-v3-base-attention_fgm/config/config.json'

weightedmean2last_fgm_1024_CFG = load_config('/kaggle/input/fb3models/20221127-065251-deberta-v3-base-weightedmean2last_fgm_1024')
weightedmean2last_fgm_1024_CFG.name = 'weightedmean2last_fgm_1024'
weightedmean2last_fgm_1024_CFG.version = 'custom'
weightedmean2last_fgm_1024_CFG.config_path = '/kaggle/input/fb3models/20221127-065251-deberta-v3-base-weightedmean2last_fgm_1024/config/config.json'

weighted2last_fgm_512_CFG = load_config('../input/fb3models/20221124-060246-deberta-v3-base-weighted2last_fgm')
weighted2last_fgm_512_CFG.name = 'weighted2last_fgm_512'
weighted2last_fgm_512_CFG.version = 'custom'
weighted2last_fgm_512_CFG.config_path = '../input/fb3models/20221124-060246-deberta-v3-base-weighted2last_fgm/config/config.json'

weightedmean2last_fgm_512_CFG = load_config('../input/fb3models/20221124-160318-deberta-v3-base-weightedmean2last_fgm')
weightedmean2last_fgm_512_CFG.name = 'weightedmean2last_fgm_512'
weightedmean2last_fgm_512_CFG.version = 'custom'
weightedmean2last_fgm_512_CFG.config_path = '../input/fb3models/20221124-160318-deberta-v3-base-weightedmean2last_fgm/config/config.json'

roberta_attention_fgm_CFG = load_config('../input/fb3models/20221121-173739-roberta-base')
roberta_attention_fgm_CFG.name = 'roberta_attention_large_fgm'
roberta_attention_fgm_CFG.version = 'custom'
roberta_attention_fgm_CFG.config_path = '../input/fb3models/20221121-173739-roberta-base/config/config.json'

##########
v112_CFG.pretrained = False
v114_CFG.pretrained = False
v116_CFG.pretrained = False
v21_CFG.pretrained = False
v2_CFG.pretrained = False
attention_fgm_CFG.pretrained = False
weighted_attention_CFG.pretrained = False
weightedpool_CFG.pretrained = False
mean_attention_no_fgm_CFG.pretrained=False
attention_large_fgm_CFG.pretrained=False
attention_fgm_512_CFG.pretrained = False
attention_fgm_768_CFG.pretrained = False
weighted2last_fgm_512_CFG.pretrained = False
weightedmean2last_fgm_512_CFG.pretrained = False
roberta_attention_fgm_CFG.pretrained = False
weightedmean2last_fgm_1024_CFG.pretrained = False


weighted_fgm_CFG.inference_weight = 1.0
v114_CFG.inference_weight = 1.0 
v2_CFG.inference_weight = 1.0 
v21_CFG.inference_weight = 1.0
attention_fgm_CFG.inference_weight = 1.0
weighted_attention_CFG.inference_weight = 1.0
attention_large_fgm_CFG.inference_weight = 1.0
attention_fgm_512_CFG.inference_weight = 1.0
attention_fgm_768_CFG.inference_weight = 1.0
weighted2last_fgm_512_CFG.inference_weight = 1.0
weightedmean2last_fgm_512_CFG.inference_weight = 1.0
roberta_attention_fgm_CFG.inference_weight = 1.0
weightedmean2last_fgm_1024_CFG.inference_weight = 1.0

In [11]:
fine_tuned_models_cfg = [
    attention_large_fgm_CFG, weighted2last_fgm_512_CFG, weightedmean2last_fgm_512_CFG, attention_fgm_512_CFG, attention_fgm_768_CFG, 
    v21_CFG, v112_CFG, mean_attention_no_fgm_CFG, v2_CFG, weightedpool_CFG, attention_fgm_CFG]

# for i,cfg in enumerate(fine_tuned_models_cfg):
#     fine_tuned_models_cfg[i].inference_weight = optimal_weights[i]
#     print(fine_tuned_models_cfg[i].name, fine_tuned_models_cfg[i].inference_weight)

In [12]:
print('Total number of models:', len(fine_tuned_models_cfg))
stacking = False

Total number of models: 11


In [13]:
test = pd.read_csv(TEST_PATH)
# test['tokenize_length'] = [len(cfg.tokenizer(text)['input_ids']) for text in test['full_text'].values]
# test = test.sort_values('tokenize_length', ascending=True).reset_index(drop=True)

In [14]:
fine_tuned_predictions = []
total_weight = 0
for cfg in tqdm(fine_tuned_models_cfg):
    infer_ = Inferencer(cfg=cfg, inference_weight=cfg.inference_weight)
    if cfg.path in [
        attention_large_fgm_CFG.path,
        attention_fgm_CFG.path, attention_fgm_768_CFG.path, roberta_attention_fgm_CFG.path, attention_fgm_512_CFG.path, 
        weighted2last_fgm_512_CFG.path, weightedmean2last_fgm_512_CFG.path, weightedmean2last_fgm_1024_CFG.path
    ]:
        file_type = 'csv'
    else:
        file_type = 'pkl'

    infer_.get_oof_result(file_type)
    
    test_dataset = TestDataset(cfg, test)
    if cfg.path == roberta_attention_fgm_CFG.path:
        collate_fn = DataCollatorWithPadding(tokenizer=cfg.tokenizer, max_length=cfg.max_len, padding='max_length')
        print('roberta=512')
    elif '512' in cfg.name:
        collate_fn = DataCollatorWithPadding(tokenizer=cfg.tokenizer, padding='max_length', max_length=512)
        print('max_len=512') 
    elif '768' in cfg.name:
        collate_fn = DataCollatorWithPadding(tokenizer=cfg.tokenizer, padding='max_length', max_length=768)
        print('max_len=768') 
    elif '1024' in cfg.name:
        collate_fn = DataCollatorWithPadding(tokenizer=cfg.tokenizer, padding='max_length', max_length=1024)
        print('max_len=1024')
    else:
        collate_fn = DataCollatorWithPadding(tokenizer=cfg.tokenizer, padding='longest')
    test_loader = DataLoader(
        test_dataset,
        batch_size=12,
        shuffle=False,
        collate_fn=collate_fn,
        #num_workers=1,
        num_workers=0,
        pin_memory=True, 
        drop_last=False)
    prediction = infer_.predict(test_loader, device) * cfg.inference_weight
    
    fine_tuned_predictions.append(prediction)
    total_weight += cfg.inference_weight
    
    del infer_, test_dataset, test_loader, prediction; gc.collect; torch.cuda.empty_cache();
    
final_fine_tuned_predictions = np.sum(fine_tuned_predictions, axis=0)/total_weight
final_fine_tuned_predictions

  0%|          | 0/11 [00:00<?, ?it/s]

score: 0.452063  scores: [0.48157374223152627, 0.4452090627651757, 0.4154085184288849, 0.4536476489584571, 0.47240178087223106, 0.44413673294015477]
########## ../input/fb3models/20221118-164148-deberta-v3-large-attention_fgm ##########
Predicting fold 0...
########## ETA: 36.16s ########## 

########## ../input/fb3models/20221118-164148-deberta-v3-large-attention_fgm ##########
Predicting fold 1...
########## ETA: 30.33s ########## 

########## ../input/fb3models/20221118-164148-deberta-v3-large-attention_fgm ##########
Predicting fold 2...
########## ETA: 31.22s ########## 

########## ../input/fb3models/20221118-164148-deberta-v3-large-attention_fgm ##########
Predicting fold 3...
########## ETA: 30.90s ########## 

score: 0.453033  scores: [0.48389889660631685, 0.44678944048963176, 0.41465519997691047, 0.45360604538329913, 0.4717536101123449, 0.4474973036716195]
max_len=512
########## ../input/fb3models/20221124-060246-deberta-v3-base-weighted2last_fgm ##########
Predicting fold 0.

array([[2.9428473, 2.8288896, 3.1413603, 3.016755 , 2.7340455, 2.7014422],
       [2.6816514, 2.4797902, 2.7202091, 2.3856783, 2.155445 , 2.627569 ],
       [3.577352 , 3.3825758, 3.56533  , 3.5409017, 3.3748686, 3.2661853]],
      dtype=float32)

In [15]:
total_weight

10.1

In [16]:
final_fine_tuned_predictions = np.clip(final_fine_tuned_predictions, 1, 5)
test[TARGET_COLS] = final_fine_tuned_predictions

# submit

In [17]:
submission = pd.read_csv(SUBMISSION_PATH)
submission = submission.drop(columns=TARGET_COLS).merge(test[['text_id'] + TARGET_COLS], on='text_id', how='left')
display(submission.head())
submission[['text_id'] + TARGET_COLS].to_csv('submission.csv', index=False)

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.942847,2.828890,3.141360,3.016755,2.734046,2.701442
1,000BAD50D026,2.681651,2.479790,2.720209,2.385678,2.155445,2.627569
2,00367BB2546B,3.577352,3.382576,3.565330,3.540902,3.374869,3.266185
